<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/earthengine-py-notebooks/tree/master/FeatureCollection/metadata_aggregation.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/earthengine-py-notebooks/blob/master/FeatureCollection/metadata_aggregation.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://mybinder.org/v2/gh/giswqs/earthengine-py-notebooks/master?filepath=FeatureCollection/metadata_aggregation.ipynb"><img width=58px src="https://mybinder.org/static/images/logo_social.png" />Run in binder</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/earthengine-py-notebooks/blob/master/FeatureCollection/metadata_aggregation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install Earth Engine API
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geehydro](https://github.com/giswqs/geehydro). The **geehydro** Python package builds on the [folium](https://github.com/python-visualization/folium) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, `Map.centerObject()`, and `Map.setOptions()`.
The following script checks if the geehydro package has been installed. If not, it will install geehydro, which automatically install its dependencies, including earthengine-api and folium.

In [1]:
import subprocess

try:
    import geehydro
except ImportError:
    print('geehydro package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geehydro'])

Import libraries

In [2]:
import ee
import folium
import geehydro

Authenticate and initialize Earth Engine API. You only need to authenticate the Earth Engine API once. 

In [3]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create an interactive map 
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [4]:
Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')

## Add Earth Engine Python script 

In [5]:
def cal_area(feature):
    num = ee.Number.parse(feature.get('areasqkm'))
    return feature.set('areasqkm', num)

# Load watersheds from a data table.
sheds = ee.FeatureCollection('USGS/WBD/2017/HUC06') \
  .filterBounds(ee.Geometry.Rectangle(-127.18, 19.39, -62.75, 51.29)) \
  .map(cal_area)

# Display the table and print its first element.
# Map.addLayer(sheds, {}, 'watersheds')
Map.addLayer(ee.Image().paint(sheds, 1, 2), {}, 'watersheds')
print('First watershed', sheds.first().getInfo())

# Print the number of watersheds.
print('Count:', sheds.size().getInfo())

# Print stats for an area property.
# print('Area stats:', sheds.aggregate_stats('areasqkm').getInfo())


First watershed {'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-92.16278604720992, 35.01926068775125], [-92.16285321427759, 35.01979763733028], [-92.16270126392283, 35.02045263533042], [-92.16240546070843, 35.02104566564045], [-92.16218282247125, 35.021580690402864], [-92.16174294652697, 35.022055554985414], [-92.16166633598948, 35.02209698368342], [-92.16180591418974, 35.022184559066964], [-92.17071027914275, 35.02163572981398], [-92.19482771864463, 35.01996793552579], [-92.20188245079504, 35.01894702235223], [-92.20410694393073, 35.0189615778368], [-92.20985355849592, 35.01899913091223], [-92.21523825931492, 35.018119022296304], [-92.21968867169772, 35.017995194246865], [-92.23453710751454, 35.016107124940554], [-92.24084245242115, 35.015841724490144], [-92.24659456612244, 35.01526747642107], [-92.24845241582662, 35.014821491905366], [-92.2519744808351, 35.01484323461566], [-92.25624619623336, 35.013954385597565], [-92.26014020872654, 35.01382568881116], [-92.2

## Display Earth Engine data layers 

In [6]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map